# IMPORT THƯ VIỆN

In [1]:
import logging
import json
from typing import List, Dict, Optional
import re
from collections import Counter
import pandas as pd

# LOGGER

In [2]:
def _setup_logger(name: str, level=logging.INFO, log_file=None) -> logging.Logger:
    '''
         Hàm khởi tạo và trả về logger đã cấu hình
    '''
    formatter = logging.Formatter('<%(levelname)s-%(name)s> - %(message)s')

    handler = logging.StreamHandler()
    handler.setFormatter(formatter)

    logger = logging.getLogger(name)
    logger.setLevel(level)

    # Tránh nhân bản handler
    if not logger.handlers:
        logger.addHandler(handler)

        # Nếu muốn ghi vào file log
        if log_file:
            file_handler = logging.FileHandler(log_file)
            file_handler.setFormatter(formatter)
            logger.addHandler(file_handler)

    return logger

In [3]:
logger_level = logging.DEBUG

# UTILS

In [22]:
utils_logger = _setup_logger("Utils", logger_level)

class Utils:
    @staticmethod
    def load_json(file_path: str) -> dict:
        """Đọc file .json và trả về dữ liệu dưới dạng dict hoặc list."""
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                utils_logger.debug(f"Load FILE JSON {file_path} thành công")
                return data
        except Exception as e:
            utils_logger.debug(f"Load FILE JSON {file_path} thất bại")
            raise Exception(e)

    @staticmethod
    def load_jsonl(file_path: str) -> List[Dict]:
        """Đọc file .jsonl và trả về list chứa các dict (mỗi dòng là một JSON).
    
        Args:
            file_path (str): Đường dẫn tới file .jsonl
            utils_logger: Logger đã được cấu hình sẵn (thường dùng logging.getLogger(...))
    
        Returns:
            List[Dict]: Danh sách các dòng JSON đã parse thành dict
        """
        data = []
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                for line_num, line in enumerate(f, start=1):
                    line = line.strip()
                    if line:
                        try:
                            data.append(json.loads(line))
                        except json.JSONDecodeError as e:
                            utils_logger.debug(f"Lỗi JSON ở dòng {line_num}: {e}")
        except FileNotFoundError:
            utils_logger.debug(f"Không tìm thấy file: {file_path}")
        except Exception as e:
            utils_logger.debug(f"Lỗi khi đọc file '{file_path}': {e}")
        return data

    @staticmethod
    def write_csv(data: List[Dict], file_path: str, encoding: str = "utf-8", index: bool = False) -> None:
        """
        Ghi list[dict] hoặc pandas.DataFrame ra file CSV.
    
        Args:
            data (List[Dict] or pd.DataFrame): Dữ liệu cần ghi.
            file_path (str): Đường dẫn file CSV đầu ra.
            encoding (str): Kiểu mã hóa (mặc định: "utf-8").
            index (bool): Có ghi chỉ số dòng (index) hay không (mặc định: False).
        """
        try:
            if isinstance(data, pd.DataFrame):
                df = data
            else:
                df = pd.DataFrame(data)
    
            df.to_csv(file_path, index=index, encoding=encoding)
            utils_logger.debug(f"[write_csv] ✅ Đã ghi {len(df)} dòng vào '{file_path}'")
    
        except Exception as e:
            utils_logger.debug(f"[write_csv] ❌ Lỗi khi ghi CSV: {e}")

# POST ENTITY

In [5]:
entity_logger = _setup_logger("Entity", logger_level)

In [6]:
class CommentReactionsInfo:
    def __init__(self, reaction_data: Dict):
        self.total = reaction_data.get("total", 0)
        self.detail = reaction_data.get("detail", {}) or {}
        self.reaction_key_map = {
            k.lower(): k for k in self.detail.keys()
        }

    def get_total(self) -> int:
        return self.total

    def get_count(self, reaction: str) -> int:
        key = self.reaction_key_map.get(reaction.lower())
        if key is None:
            entity_logger.debug(f"[CommentReactionsInfo] '{reaction}' không tồn tại trong reaction_key_map")
            return 0
        return self.detail.get(key, 0)

    def get_percentage(self, reaction: str) -> float:
        count = self.get_count(reaction)
        return (count / self.total) * 100 if self.total > 0 else 0.0

    def get_all_counts(self) -> Dict[str, int]:
        return dict(self.detail)

    def get_all_percentages(self) -> Dict[str, float]:
        return {
            k: round((v / self.total) * 100, 2) if self.total > 0 else 0.0
            for k, v in self.detail.items()
        }

    def most_common(self, top_n: int = 2) -> List[tuple]:
        return sorted(self.detail.items(), key=lambda x: x[1], reverse=True)[:top_n]

    def get_available_reactions(self) -> List[str]:
        return list(self.detail.keys())

In [7]:
class ParentComment:
    def __init__(self, data: Dict):
        self.text = data.get("text", "")
        self.reactions_raw = data.get("reactions", {})
        self.raw = data

        entity_logger.debug(f"[ParentComment] Init | Length: {self.length()}")

    def get_text(self) -> str:
        return self.text

    def has_link(self) -> bool:
        has = "http" in self.text
        entity_logger.debug(f"[ParentComment] has_link: {has}")
        return has

    def length(self) -> int:
        return len(self.text)

    def get_keywords(self) -> List[str]:
        words = re.findall(r'\w+', self.text.lower())
        entity_logger.debug(f"[ParentComment] {len(words)} keywords")
        return words

    def get_reactions_info(self) -> CommentReactionsInfo:
        entity_logger.debug("[ParentComment] get_reactions_info")
        return CommentReactionsInfo(self.reactions_raw)

    def get_reaction_count(self) -> int:
        info = self.get_reactions_info()
        return info.get_total()

    def to_dict(self) -> Dict:
        return {
            "text": self.text,
            "reaction_count": self.get_reaction_count(),
            "has_link": self.has_link(),
            "length": self.length(),
        }


In [8]:
class ReactionsInfo:
    def __init__(self, reaction_dict: Dict[str, int]):
        self.reactions = reaction_dict or {}
        self.total = sum(self.reactions.values())       

    def get_total(self) -> int:
        return self.total

    def get_count(self, reaction: str) -> int:
        """Trả về số lượng reaction theo tên không phân biệt hoa thường"""
        key = reaction
        if key is None:
            entity_logger.debug(f"[get_count] '{reaction}' không tồn tại trong reaction_key_map")
            return 0
        return self.reactions.get(key, 0)

    def get_percentage(self, reaction: str) -> float:
        count = self.get_count(reaction)
        return (count / self.total) * 100 if self.total > 0 else 0.0

    def get_all_counts(self) -> Dict[str, int]:
        return dict(self.reactions)

    def get_all_percentages(self) -> Dict[str, float]:
        return {
            k: round((v / self.total) * 100, 2) if self.total > 0 else 0.0
            for k, v in self.reactions.items()
        }

    def most_common(self, top_n: int = 3) -> List[tuple]:
        return sorted(self.reactions.items(), key=lambda x: x[1], reverse=True)[:top_n]

    def get_available_reactions(self) -> List[str]:
        """Trả về danh sách các loại cảm xúc có mặt trong post"""
        return list(self.reactions.keys())

In [9]:
class FacebookPost:
    def __init__(self, data: Dict):
        entity_logger.debug("[FacebookPost] Initializing...")
        self.data = data
        
        raw_comments = data.get("comments", {}).get("comments", [])
        entity_logger.debug(f"[FacebookPost] Found {len(raw_comments)} parent comments.")
        self.parent_comments: List[ParentComment] = [
            ParentComment(c) for c in raw_comments
        ]

    def get_creation_time(self) -> Optional[str]:
        ts = self.data.get("creation_time")
        if ts:
            try:
                dt = datetime.utcfromtimestamp(ts)
                formatted = dt.strftime('%Y-%m-%d %H:%M:%S UTC')
                entity_logger.debug(f"[get_creation_time] Formatted time: {formatted}")
                return formatted
            except Exception as e:
                entity_logger.debug(f"[get_creation_time] Error formatting timestamp: {e}")
        return None

    def get_reactions_info(self) -> ReactionsInfo:
        reaction_data = self.data.get("reactions_detail", {})
        entity_logger.debug(f"[get_reactions_info] Raw: {reaction_data}")
        return ReactionsInfo(reaction_data)

    def get_post_text(self) -> str:
        text = self.data.get("post_content", "")
        entity_logger.debug(f"[get_post_text] Length: {len(text)}")
        return text

    def get_post_url(self) -> str:
        return self.data.get("post_url", "")

    def get_total_reactions(self) -> int:
        total = self.data.get("total_reactions", 0)
        entity_logger.debug(f"[get_total_reactions] Total: {total}")
        return total

    def get_reaction_breakdown(self) -> Dict[str, int]:
        reactions = self.data.get("reactions_detail", {})
        entity_logger.debug(f"[get_reaction_breakdown] Reactions: {reactions}")
        return reactions

    def get_comment_count(self) -> int:
        count = self.data.get("comment_count", 0)
        entity_logger.debug(f"[get_comment_count] Total: {count}")
        return count

    def get_share_count(self) -> int:
        try:
            share_count = int(self.data.get("share_count", "0"))
        except ValueError:
            share_count = 0
        entity_logger.debug(f"[get_share_count] Share count: {share_count}")
        return share_count

    def get_parent_comments(self) -> List[ParentComment]:
        entity_logger.debug(f"[get_parent_comments] {len(self.parent_comments)} bình luận cha")
        return self.parent_comments

    def get_average_comment_length(self) -> float:
        if not self.parent_comments:
            return 0.0
        lengths = [c.length() for c in self.parent_comments]
        avg = sum(lengths) / len(lengths)
        entity_logger.debug(f"[get_average_comment_length] Trung bình: {avg}")
        return avg

    def get_link_comments(self) -> List[ParentComment]:
        linked = [c for c in self.parent_comments if c.has_link()]
        entity_logger.debug(f"[get_link_comments] Có {len(linked)} bình luận chứa link")
        return linked

    def get_top_keywords_in_comments(self, top_n: int = 10) -> List[tuple]:
        counter = Counter()
        for c in self.parent_comments:
            counter.update(c.get_keywords())
        result = counter.most_common(top_n)
        entity_logger.debug(f"[get_top_keywords_in_comments] Top {top_n}: {result}")
        return result

    def get_first_comment_with_link(self) -> Optional[ParentComment]:
        for c in self.parent_comments:
            if c.has_link():
                entity_logger.debug("[get_first_comment_with_link] Tìm thấy bình luận có link")
                return c
        entity_logger.debug("[get_first_comment_with_link] Không có bình luận nào chứa link")
        return None

# PREPARE DATA 

In [10]:
prepare_logger = _setup_logger("PrepareData", logger_level)

In [20]:
class PrepareData:
    @staticmethod
    def prepare_text_each_post(post: FacebookPost, max_comment: int = 50) -> List[str]:
        """
        Thực hiện các bước:
        Bước 1: Lấy nội dung bài post (post content)
        Bước 2: Lấy danh sách parent comment và nội dung từng comment
        Bước 3: Với mỗi comment, ghép nội dung post + comment thành một chuỗi → trả về list
        """
        # Bước 1: lấy nội dung post
        post_text = post.get_post_text().strip()
        prepare_logger.debug(f"[Bước 1] Post content: '{post_text[:80]}...' (length: {len(post_text)})")

        # Bước 2: lấy comment cha
        parent_comments = post.get_parent_comments()
        prepare_logger.debug(f"[Bước 2] Số lượng comment cha: {len(parent_comments)}")

        comment_texts = []
        n_comment = len(parent_comments)
        top_k_cmt = min(n_comment, max_comment)
        for i, c in enumerate(parent_comments[:top_k_cmt]):
            text = c.get_text().strip()
            if text:
                comment_texts.append(text)
                prepare_logger.debug(f"  - Comment #{i+1} | length: {len(text)} | preview: '{text[:60]}...'")

        # Bước 3: ghép post + comment
        merged_texts = [f"{post_text}\n\n{comment}" for comment in comment_texts]
        prepare_logger.debug(f"[Bước 3] Tổng số mẫu ghép: {len(merged_texts)}")

        return merged_texts

    @staticmethod
    def prepare_corpus_content_df(posts: List[FacebookPost], max_comment: int = 50) -> pd.DataFrame:
        """
        Trả về DataFrame gồm 2 cột: 'content' và 'post_id'
        - 'content': văn bản sau khi merge post content + comment content
        - 'post_id': lấy từ 'feedback_id' trong dict gốc
        """
        prepare_logger.debug(f"[prepare_corpus_content_df] Xử lý {len(posts)} bài viết | Max Comment: {max_comment}")
    
        records = []
    
        for i, post in enumerate(posts):
            try:
                prepare_logger.debug(f"[Post {i+1}] Đang xử lý post")
                post_id = i + 1
                merged_texts = PrepareData.prepare_text_each_post(post, max_comment)
    
                for content in merged_texts:
                    records.append({
                        "post_id": post_id,
                        "content": content
                    })
    
                prepare_logger.debug(f"[Post {i+1}] Thêm {len(merged_texts)} dòng vào DataFrame")
    
            except Exception as e:
                prepare_logger.debug(f"[Post {i+1}] Lỗi: {e}")
    
        df = pd.DataFrame(records)
        prepare_logger.debug(f"[prepare_corpus_content_df] Tổng số dòng: {len(df)}")
        return df

# MAIN 

## Load Posts

In [12]:
logger = _setup_logger("Main", logger_level)

In [13]:
post_file = "/kaggle/input/the-anh-post/posts_Theanh28.json"
posts_list = Utils.load_json(post_file)
logger.debug(f"Length post: {len(posts_list)}")

<DEBUG-Utils> - Load FILE JSON /kaggle/input/the-anh-post/posts_Theanh28.json thành công
<DEBUG-Main> - Length post: 135


In [14]:
posts_entity = [FacebookPost(p) for p in posts_list]
logger.debug(len(posts_entity))

<DEBUG-Entity> - [FacebookPost] Initializing...
<DEBUG-Entity> - [FacebookPost] Found 50 parent comments.
<DEBUG-Entity> - [ParentComment] Init | Length: 196
<DEBUG-Entity> - [ParentComment] Init | Length: 99
<DEBUG-Entity> - [ParentComment] Init | Length: 39
<DEBUG-Entity> - [ParentComment] Init | Length: 82
<DEBUG-Entity> - [ParentComment] Init | Length: 66
<DEBUG-Entity> - [ParentComment] Init | Length: 33
<DEBUG-Entity> - [ParentComment] Init | Length: 36
<DEBUG-Entity> - [ParentComment] Init | Length: 36
<DEBUG-Entity> - [ParentComment] Init | Length: 46
<DEBUG-Entity> - [ParentComment] Init | Length: 233
<DEBUG-Entity> - [ParentComment] Init | Length: 54
<DEBUG-Entity> - [ParentComment] Init | Length: 53
<DEBUG-Entity> - [ParentComment] Init | Length: 47
<DEBUG-Entity> - [ParentComment] Init | Length: 86
<DEBUG-Entity> - [ParentComment] Init | Length: 109
<DEBUG-Entity> - [ParentComment] Init | Length: 9
<DEBUG-Entity> - [ParentComment] Init | Length: 10
<DEBUG-Entity> - [ParentC

In [ ]:
df = PrepareData.prepare_corpus_content_df(posts_entity, 100)

<DEBUG-PrepareData> - [prepare_corpus_content_df] Xử lý 135 bài viết | Max Comment: 5
<DEBUG-PrepareData> - [Post 1] Đang xử lý post
<DEBUG-Entity> - [get_post_text] Length: 67
<DEBUG-PrepareData> - [Bước 1] Post content: 'Quảng Ninh: Cô gái 16 tuổi suy gan sau khi giảm cân không đúng cách...' (length: 67)
<DEBUG-Entity> - [get_parent_comments] 50 bình luận cha
<DEBUG-PrepareData> - [Bước 2] Số lượng comment cha: 50
<DEBUG-PrepareData> -   - Comment #1 | length: 196 | preview: 'Cô được chẩn đoán rối loạn chức năng gan nghiêm trọng, men g...'
<DEBUG-PrepareData> -   - Comment #2 | length: 99 | preview: 'Tập Gym là chân ái, vừa khỏe vừa an toàn 😄 em bảo đi tập thì...'
<DEBUG-PrepareData> -   - Comment #3 | length: 39 | preview: ':))))) mong mẹ tôi kh nhìn thấy bài này...'
<DEBUG-PrepareData> -   - Comment #4 | length: 82 | preview: 'sao giừ còn uống thuốc giảm cân được zị 🥲🥲 ăn heathy thâm hụ...'
<DEBUG-PrepareData> -   - Comment #5 | length: 66 | preview: '4.5 tháng giảm dc 18 kg nè...b

In [ ]:
data_file = "content_posts.csv"
Utils.write_csv(df, data_file)